In [3]:
import os
import re

from traits.trait_types import self

# ------------------------获取x,y的size------------------------
parameter_file = 'Noc_parameters.v'
reg_file = open(parameter_file)
parameters_value_list = []
parameters_name_list = []
for line in reg_file.readlines():
    m = re.match(r"`define(\s+)", line)
    if m:
        line = line[len(m.group(0)):]
        m = re.match(r"(\S+)", line)
        if m:
            line = line[len(m.group(0)):]
            parameters_name = m.group(0)
            if line == '\n':
                continue
            m = re.match(r"\s+", line)
            if m:
                s = len(m.group(0))
                try:
                    a = int(line[s:])
                    parameters_name_list.append(parameters_name)
                    parameters_value_list.append(a)
                except ValueError:
                    continue

Noc_X_Size = 0
Noc_Y_Size = 0
Noc_channel = 0
for index, name in enumerate(parameters_name_list):
    if name == "Noc_X_Size":
        Noc_X_Size = parameters_value_list[index]
    elif name == "Noc_Y_Size":
        Noc_Y_Size = parameters_value_list[index]
    elif name == "Noc_VC_Channel":
        Noc_channel = parameters_value_list[index]
if Noc_X_Size == 0 or Noc_Y_Size == 0:
    print("找不到Noc_X_Size,Noc_Y_Size")

In [4]:
# ------------------------------- system verilog interface To verilog interface ----------------------------
code_str = '''/*
    created by: <Xidian University>
    created date: 2024-05-16
*/

module Noc_fabric_verilog_interface import Noc_parameters::*;
(
    input                           noc_clk,
    input                           noc_rst_n,

'''
for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        for c in range(Noc_channel):
            port_str = '''    input   [0:0]                   Noc_{x}_{y}_channel{c}_receive_valid,
    output  [0:0]                   Noc_{x}_{y}_channel{c}_receive_ready,
    input   [Noc_Data_Width-1:0]    Noc_{x}_{y}_channel{c}_receive_flit,
    output  [0:0]                   Noc_{x}_{y}_channel{c}_receive_vc_ready,
    input   [0:0]                   Noc_{x}_{y}_channel{c}_receive_is_header,
    input   [0:0]                   Noc_{x}_{y}_channel{c}_receive_is_tail,
'''.format(x=x, y=y, c=c)
            code_str += port_str

        port_str = ''' 
    output  [0:0]                   Noc_{x}_{y}_sender_valid,
    input   [0:0]                   Noc_{x}_{y}_sender_ready,
    output  [Noc_Data_Width-1:0]    Noc_{x}_{y}_sender_flit,
    input   [0:0]                   Noc_{x}_{y}_sender_vc_ready,
    output  [0:0]                   Noc_{x}_{y}_sender_is_header,
    output  [0:0]                   Noc_{x}_{y}_sender_is_tail,'''.format(x=x, y=y)
        if x == Noc_X_Size - 1 and y == Noc_Y_Size - 1:
            port_str = port_str[:-1] + '\n);'
        else:
            port_str = port_str + "\n\n"
        code_str += port_str

define_str = '''\n
Noc_flit_interface #(.Port_Type(LOCAL)) noc_receiver_if[Noc_Node_Num]();
Noc_flit_interface #(.Channel(1))       noc_sender_if[Noc_Node_Num]();'''

code_str += define_str

connect_str = ''
for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        node_index = Noc_X_Size * y + x
        for c in range(Noc_channel):
            connect_str_port = '''  
assign noc_receiver_if[{node_index}].valid[{c}] = Noc_{x}_{y}_channel{c}_receive_valid;
assign Noc_{x}_{y}_channel{c}_receive_ready = noc_receiver_if[{node_index}].ready[{c}];
assign noc_receiver_if[{node_index}].flit[{c}] = {{Noc_{x}_{y}_channel{c}_receive_is_header, Noc_{x}_{y}_channel{c}_receive_is_tail, Noc_{x}_{y}_channel{c}_receive_flit}};
assign Noc_{x}_{y}_channel{c}_receive_vc_ready = noc_receiver_if[{node_index}].vc_ready[{c}];
    '''.format(node_index=node_index, x=x, y=y, c=c)
            connect_str += connect_str_port

        connect_str_port = '''
assign Noc_{x}_{y}_sender_valid = noc_sender_if[{node_index}].valid[0];
assign noc_sender_if[{node_index}].ready[0] = Noc_{x}_{y}_sender_ready;
assign Noc_{x}_{y}_sender_flit = noc_sender_if[{node_index}].flit[0][Noc_Data_Width-1:{node_index}];
assign noc_sender_if[{node_index}].vc_ready[0] = Noc_{x}_{y}_sender_vc_ready;
assign Noc_{x}_{y}_sender_is_header = noc_sender_if[{node_index}].flit[0][Noc_Data_Width+1];
assign Noc_{x}_{y}_sender_is_tail = noc_sender_if[{node_index}].flit[0][Noc_Data_Width];
'''.format(x=x, y=y, node_index=node_index)
        connect_str += connect_str_port
connect_str += "\n\n"

code_str += connect_str

instance_str = '''Noc_fabric Noc_fabric_inst(
    .noc_clk            (noc_clk),
    .noc_rst_n          (noc_rst_n),
    .noc_receiver_if    (noc_receiver_if),
    .noc_sender_if      (noc_sender_if)
);'''

code_str += instance_str
code_str += "\n\nendmodule"

file = open("./Noc_fabric_verilog_interface.sv", 'w')
file.write(code_str)
file.close()

In [5]:
# --------------------------------- system file To verilog file -----------------------------------
code_str = '''/*
    created by: <Xidian University>
    created date: 2024-05-16
*/

`include "Noc_parameters.v"

module Noc_fabric_verilog_top
(
    input                           noc_clk,
    input                           noc_rst_n,

'''
for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        for c in range(Noc_channel):
            port_str = '''    input   [0:0]                   Noc_{x}_{y}_channel{c}_receive_valid,
    output  [0:0]                   Noc_{x}_{y}_channel{c}_receive_ready,
    input   [`Noc_Data_Width-1:0]   Noc_{x}_{y}_channel{c}_receive_flit,
    output  [0:0]                   Noc_{x}_{y}_channel{c}_receive_VCready,
    input   [0:0]                   Noc_{x}_{y}_channel{c}_receive_is_header,
    input   [0:0]                   Noc_{x}_{y}_channel{c}_receive_is_tail,
'''.format(x=x, y=y, c=c)
            code_str += port_str

        port_str = '''
    output  [0:0]                   Noc_{x}_{y}_sender_valid,
    input   [0:0]                   Noc_{x}_{y}_sender_ready,
    output  [`Noc_Data_Width-1:0]   Noc_{x}_{y}_sender_flit,
    input   [0:0]                   Noc_{x}_{y}_sender_VCready,
    output  [0:0]                   Noc_{x}_{y}_sender_is_header,
    output  [0:0]                   Noc_{x}_{y}_sender_is_tail,'''.format(x=x, y=y)
        if x == Noc_X_Size - 1 and y == Noc_Y_Size - 1:
            port_str = port_str[:-1] + '\n);'
        else:
            port_str = port_str + "\n\n"
        code_str += port_str

instance_str = '''\n\nNoc_fabric_verilog_interface Noc_fabric_verilog_interface_inst(\n'''
instance_str += '''    .noc_clk                             (noc_clk),
    .noc_rst_n                           (noc_rst_n),
'''
for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        for c in range(Noc_channel):
            instance_str_post = '''    .Noc_{x}_{y}_channel{c}_receive_valid      (Noc_{x}_{y}_channel{c}_receive_valid),
    .Noc_{x}_{y}_channel{c}_receive_ready      (Noc_{x}_{y}_channel{c}_receive_ready),
    .Noc_{x}_{y}_channel{c}_receive_flit       (Noc_{x}_{y}_channel{c}_receive_flit),
    .Noc_{x}_{y}_channel{c}_receive_vc_ready   (Noc_{x}_{y}_channel{c}_receive_VCready),
    .Noc_{x}_{y}_channel{c}_receive_is_header  (Noc_{x}_{y}_channel{c}_receive_is_header),
    .Noc_{x}_{y}_channel{c}_receive_is_tail    (Noc_{x}_{y}_channel{c}_receive_is_tail),
'''.format(x=x, y=y, c=c)
            instance_str += instance_str_post

        instance_str_post = '''    
    .Noc_{x}_{y}_sender_valid                (Noc_{x}_{y}_sender_valid),
    .Noc_{x}_{y}_sender_ready                (Noc_{x}_{y}_sender_ready),
    .Noc_{x}_{y}_sender_flit                 (Noc_{x}_{y}_sender_flit),
    .Noc_{x}_{y}_sender_vc_ready             (Noc_{x}_{y}_sender_VCready),
    .Noc_{x}_{y}_sender_is_header            (Noc_{x}_{y}_sender_is_header),
    .Noc_{x}_{y}_sender_is_tail              (Noc_{x}_{y}_sender_is_tail),'''.format(x=x, y=y)
        if x == Noc_X_Size - 1 and y == Noc_Y_Size - 1:
            instance_str_post = instance_str_post[:-1] + '\n);'
        else:
            instance_str_post = instance_str_post + "\n\n"
        instance_str += instance_str_post

code_str += instance_str
code_str += "\n\nendmodule"

file = open("./Noc_fabric_verilog_top.v", 'w')
file.write(code_str)
file.close()

In [6]:
# ------------------------------------------- 输入虚拟通道分配模块 -----------------------------------------------------
code_str = '''/*
    created by: <Xidian University>
    created date: 2024-05-16
*/

`include "Noc_parameters.v"

module Noc_vc_allocate
(
    input                           noc_clk,
    input                           noc_rst_n,
    input   [0:0]                   Noc_receive_valid,
    output  [0:0]                   Noc_receive_ready,
    input   [`Noc_Data_Width-1:0]   Noc_receive_flit,
    input   [0:0]                   Noc_receive_is_header,
    input   [0:0]                   Noc_receive_is_tail,
'''
for c in range(Noc_channel):
    port_str = '''    output  [0:0]                   Noc_channel{c}_sender_valid,
    input   [0:0]                   Noc_channel{c}_sender_ready,
    output  [`Noc_Data_Width-1:0]   Noc_channel{c}_sender_flit,
    input   [0:0]                   Noc_channel{c}_sender_VCready,
    output  [0:0]                   Noc_channel{c}_sender_is_header,
    output  [0:0]                   Noc_channel{c}_sender_is_tail,'''.format(c=c)
    if c == Noc_channel - 1:
        port_str = port_str[:-1] + '\n);'
    else:
        port_str = port_str + "\n\n"
    code_str += port_str

code_str += '''
reg [`Noc_VC_Channel-1:0]   now_select_reg;

always @(posedge noc_clk or negedge noc_rst_n)begin                                                  
    if(!noc_rst_n) begin
        now_select_reg <= 1;
    end  
    else if (Noc_receive_is_tail)begin                                                                                
        if (Noc_channel0_sender_VCready) begin
            now_select_reg <= 1<<0;
        end
'''
for c in range(Noc_channel - 1):
    port_str = '''        else if (Noc_channel{c}_sender_VCready)begin
            now_select_reg <= 1<<{c};
        end
    '''.format(c=c + 1)
    code_str += port_str

code_str += '''end                                                                
end'''

code_str += '''
assign Noc_receive_ready = (now_select_reg[0]) ? Noc_channel0_sender_VCready :'''
if Noc_channel == 1:
    code_str += ''' 0;

'''
else:
    for c in range(Noc_channel - 1):
        port_str = '''
                            (now_select_reg[{c}]) ? Noc_channel{c}_sender_ready :'''.format(c=c + 1)
        code_str += port_str
    code_str += ''' 0;

'''

for c in range(Noc_channel):
    port_str = '''assign Noc_channel{c}_sender_valid        = (now_select_reg[{c}]) ? Noc_receive_valid : 0;
assign Noc_channel{c}_sender_flit         = (now_select_reg[{c}]) ? Noc_receive_flit : 0;
assign Noc_channel{c}_sender_is_header    = (now_select_reg[{c}]) ? Noc_receive_is_header : 0;
assign Noc_channel{c}_sender_is_tail      = (now_select_reg[{c}]) ? Noc_receive_is_tail : 0;
'''.format(c=c)
    code_str += port_str

code_str += "\n\nendmodule"

file = open("./Noc_vc_allocate.v", 'w')
file.write(code_str)
file.close()

In [7]:
# ------------------------------------------- connector 模块 -----------------------------------------------------
code_str = '''/*
    created by: <Xidian University>
    created date: 2024-05-16
*/

`include "Noc_parameters.v"

module Noc_connector
(
    input                           noc_clk,
    input                           noc_rst_n,
'''

for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        port_str = '''
    input   [0:0]                   Noc_{x}_{y}_receive_valid,
    output  [0:0]                   Noc_{x}_{y}_receive_ready,
    input   [`Noc_Data_Width-1:0]   Noc_{x}_{y}_receive_flit,
    input   [0:0]                   Noc_{x}_{y}_receive_is_header,
    input   [0:0]                   Noc_{x}_{y}_receive_is_tail,
    
    output  [0:0]                   Noc_{x}_{y}_sender_valid,
    input   [0:0]                   Noc_{x}_{y}_sender_ready,
    output  [`Noc_Data_Width-1:0]   Noc_{x}_{y}_sender_flit,
    output  [0:0]                   Noc_{x}_{y}_sender_is_header,
    output  [0:0]                   Noc_{x}_{y}_sender_is_tail,'''.format(x=x, y=y)
        if x == Noc_X_Size - 1 and y == Noc_Y_Size - 1:
            port_str = port_str[:-1] + '\n);'
        else:
            port_str = port_str + "\n"
        code_str += port_str
        

wire_str = ''''''
# 设置线变量
for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        for c in range(Noc_channel):
            wire_str += '''    
wire  [0:0]                   Noc_{x}_{y}_channel{c}_receive_valid;            
wire  [0:0]                   Noc_{x}_{y}_channel{c}_receive_ready;            
wire  [`Noc_Data_Width-1:0]   Noc_{x}_{y}_channel{c}_receive_flit;
wire  [0:0]                   Noc_{x}_{y}_channel{c}_receive_VCready;                     
wire  [0:0]                   Noc_{x}_{y}_channel{c}_receive_is_header;        
wire  [0:0]                   Noc_{x}_{y}_channel{c}_receive_is_tail;          
'''.format(x=x, y=y, c=c)

code_str += wire_str

# 例化bridge模块
for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        instance_str = '''\n\n\nNoc_vc_allocate Noc_vc_allocate_{x}_{y}_instance(
    .noc_clk                                  (noc_clk                              ),
    .noc_rst_n                                (noc_rst_n                            ),
    .Noc_receive_valid                        (Noc_{x}_{y}_receive_valid                ),
    .Noc_receive_ready                        (Noc_{x}_{y}_receive_ready                ),
    .Noc_receive_flit                         (Noc_{x}_{y}_receive_flit                 ),
    .Noc_receive_is_header                    (Noc_{x}_{y}_receive_is_header            ),
    .Noc_receive_is_tail                      (Noc_{x}_{y}_receive_is_tail              ),
    '''.format(x=x, y=y)
        for c in range(Noc_channel):
            instance_str_post = '''    
    .Noc_channel{c}_sender_valid                (Noc_{x}_{y}_channel{c}_receive_valid       ),
    .Noc_channel{c}_sender_ready                (Noc_{x}_{y}_channel{c}_receive_ready       ),
    .Noc_channel{c}_sender_flit                 (Noc_{x}_{y}_channel{c}_receive_flit        ),
    .Noc_channel{c}_sender_VCready              (Noc_{x}_{y}_channel{c}_receive_VCready     ),
    .Noc_channel{c}_sender_is_header            (Noc_{x}_{y}_channel{c}_receive_is_header   ),
    .Noc_channel{c}_sender_is_tail              (Noc_{x}_{y}_channel{c}_receive_is_tail     ),'''.format(x=x, y=y, c=c)
            if c == Noc_channel - 1:
                instance_str_post = instance_str_post[:-1] + '\n);'
            else:
                instance_str_post = instance_str_post + "\n"
            instance_str += instance_str_post
        code_str += instance_str

instance_str = '''\n\n\nNoc_fabric_verilog_top Noc_fabric_verilog_top_inst(
    .noc_clk                                    (noc_clk),
    .noc_rst_n                                  (noc_rst_n),'''
for x in range(Noc_X_Size):
    for y in range(Noc_Y_Size):
        for c in range(Noc_channel):
            instance_str_post = '''    
    .Noc_{x}_{y}_channel{c}_receive_valid             (Noc_{x}_{y}_channel{c}_receive_valid     ),
    .Noc_{x}_{y}_channel{c}_receive_ready             (Noc_{x}_{y}_channel{c}_receive_ready     ),
    .Noc_{x}_{y}_channel{c}_receive_flit              (Noc_{x}_{y}_channel{c}_receive_flit      ),
    .Noc_{x}_{y}_channel{c}_receive_VCready           (Noc_{x}_{y}_channel{c}_receive_VCready   ),
    .Noc_{x}_{y}_channel{c}_receive_is_header         (Noc_{x}_{y}_channel{c}_receive_is_header ),
    .Noc_{x}_{y}_channel{c}_receive_is_tail           (Noc_{x}_{y}_channel{c}_receive_is_tail   ),
'''.format(x=x, y=y, c=c)
            instance_str += instance_str_post

        instance_str_post = ('''
    .Noc_{x}_{y}_sender_valid                       (Noc_{x}_{y}_sender_valid               ),
    .Noc_{x}_{y}_sender_ready                       (Noc_{x}_{y}_sender_ready               ),
    .Noc_{x}_{y}_sender_flit                        (Noc_{x}_{y}_sender_flit                ),
    .Noc_{x}_{y}_sender_VCready                     (1'b1                                ),
    .Noc_{x}_{y}_sender_is_header                   (Noc_{x}_{y}_sender_is_header           ),
    .Noc_{x}_{y}_sender_is_tail                     (Noc_{x}_{y}_sender_is_tail             ),'''.format(x=x, y=y))
        if x == Noc_X_Size - 1 and y == Noc_Y_Size - 1:
            instance_str_post = instance_str_post[:-1] + '\n);'
        else:
            instance_str_post = instance_str_post + "\n"
        instance_str += instance_str_post

code_str += instance_str

code_str += "\n\nendmodule"

file = open("./Noc_connector.v", 'w')
file.write(code_str)
file.close()